## Obtaining camera Intrinsics

In [4]:
import cv2
import numpy as np
import glob
import os

def calibrate_chessboard_images(image_folder, pattern_size=(6, 8), square_size=25.0):
    """
    Calibrate camera given a set of chessboard images.
    
    :param image_folder: Folder containing chessboard images.
    :param pattern_size: Tuple of (number_of_corners_along_width, number_of_corners_along_height).
    :param square_size: Physical size of each chessboard square (in cm, mm, or any consistent unit).
    :return: camera_matrix, dist_coeffs, rvecs, tvecs, error
    """

    # Termination criteria for corner refinement
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Prepare 3D object points for a single chessboard
    objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

    # Scale object points by the size of each square
    objp *= square_size

    # Arrays to store object points and image points from all images
    objpoints = []  # 3D points in real-world space
    imgpoints = []  # 2D points in image plane

    images = glob.glob(os.path.join(image_folder, '*_Color.png'))  # or '*.png', etc.
    if not images:
        print(f"No images found in {image_folder}")
        return None, None, None, None, None

    # Track image shape (width, height)
    image_shape = None

    for fname in images:
        img = cv2.imread(fname)
        # If the image can't be read, skip it
        if img is None:
            print(f"Could not read {fname}. Skipping...")
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # We set image_shape if not already set
        if image_shape is None:
            # OpenCV calibrateCamera expects (width, height) order
            image_shape = gray.shape[::-1]

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)

        if ret:
            # Refine corner positions to sub-pixel accuracy
            corners_refined = cv2.cornerSubPix(
                gray, corners, (11, 11), (-1, -1), criteria
            )

            objpoints.append(objp)
            imgpoints.append(corners_refined)

            # Draw corners (optional for visualization)
            cv2.drawChessboardCorners(img, pattern_size, corners_refined, ret)
            cv2.imshow('Chessboard Corners', img)
            cv2.waitKey()  # Show each result briefly

    cv2.destroyAllWindows()

    # If we never detected corners, there is nothing to calibrate
    if not objpoints or not imgpoints:
        print("No corners were detected in any image. Calibration aborted.")
        return None, None, None, None, None

    # Perform camera calibration
    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
        objpoints,
        imgpoints,
        image_shape,  # Using stored shape
        None,
        None
    )

    # Compute overall reprojection error
    total_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(
            objpoints[i], rvecs[i], tvecs[i], camera_matrix, dist_coeffs
        )
        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
        total_error += error

    mean_error = total_error / len(objpoints)

    print("Camera matrix:\n", camera_matrix)
    print("Distortion coefficients:", dist_coeffs.ravel())
    print("Mean reprojection error:", mean_error)

    return camera_matrix, dist_coeffs, rvecs, tvecs, mean_error


In [2]:
print(os.getcwd()) # print current working directory

c:\Users\nadil\Desktop\FalconE-F1-Tenth\Labs\lab7_vision_lab


In [5]:
folder = "Resources\calibration"
calibrate_chessboard_images(folder, pattern_size=(6, 8), square_size=25.0)

[[  0.   0.   0.]
 [ 25.   0.   0.]
 [ 50.   0.   0.]
 [ 75.   0.   0.]
 [100.   0.   0.]
 [125.   0.   0.]
 [  0.  25.   0.]
 [ 25.  25.   0.]
 [ 50.  25.   0.]
 [ 75.  25.   0.]
 [100.  25.   0.]
 [125.  25.   0.]
 [  0.  50.   0.]
 [ 25.  50.   0.]
 [ 50.  50.   0.]
 [ 75.  50.   0.]
 [100.  50.   0.]
 [125.  50.   0.]
 [  0.  75.   0.]
 [ 25.  75.   0.]
 [ 50.  75.   0.]
 [ 75.  75.   0.]
 [100.  75.   0.]
 [125.  75.   0.]
 [  0. 100.   0.]
 [ 25. 100.   0.]
 [ 50. 100.   0.]
 [ 75. 100.   0.]
 [100. 100.   0.]
 [125. 100.   0.]
 [  0. 125.   0.]
 [ 25. 125.   0.]
 [ 50. 125.   0.]
 [ 75. 125.   0.]
 [100. 125.   0.]
 [125. 125.   0.]
 [  0. 150.   0.]
 [ 25. 150.   0.]
 [ 50. 150.   0.]
 [ 75. 150.   0.]
 [100. 150.   0.]
 [125. 150.   0.]
 [  0. 175.   0.]
 [ 25. 175.   0.]
 [ 50. 175.   0.]
 [ 75. 175.   0.]
 [100. 175.   0.]
 [125. 175.   0.]]
Camera matrix:
 [[694.71543755   0.         449.37542153]
 [  0.         695.54962013 258.64702588]
 [  0.           0.           1.   

(array([[694.71543755,   0.        , 449.37542153],
        [  0.        , 695.54962013, 258.64702588],
        [  0.        ,   0.        ,   1.        ]]),
 array([[ 0.14754686,  0.19250433, -0.0091839 , -0.01212939, -1.70648829]]),
 (array([[-0.55996241],
         [-0.0215673 ],
         [ 0.009381  ]]),
  array([[-0.9344358 ],
         [ 0.07726831],
         [ 0.0483663 ]]),
  array([[ 0.63273752],
         [ 0.15823964],
         [-0.06299979]]),
  array([[ 0.94182128],
         [-0.29075675],
         [-0.4049744 ]]),
  array([[-0.84928176],
         [ 0.73402417],
         [ 0.3218125 ]]),
  array([[-0.04262692],
         [ 0.08645932],
         [-0.00041802]]),
  array([[-0.03495156],
         [ 0.65769313],
         [-0.02085981]]),
  array([[-0.03235605],
         [ 1.0217236 ],
         [-0.03498579]]),
  array([[-0.03638613],
         [ 1.2576142 ],
         [-0.04471628]]),
  array([[0.53608633],
         [0.65327502],
         [1.47922949]]),
  array([[-0.04347484],
    